# Experiments

This notebook contains all the experiments conducted for the paper

## Experiment Set 1

### This experiment set is to test the results using the smaller test dataset of 5 annotators (including 1 expert). Results are reproduced from the development of the crowd yolo model, compared to other models using the same dataset.

The dataset used is a small dataset of 100 radiographs with 5 annotators (500 sets of labels) one of which is compleeted by a dental expert. The expert labels are separated and used as ground truth labels for the test dataset.


There are 4 models/techniques to expeiment:
- `Crowd Yolo`: A combination of YoloV5 object detection algorithm and a bayesian classifier combination network BCCNet (a network that decides the annotators value as an annotator inorder to train the yolo model). (main model the paper is about)
- `Crowd RCNN`: Another crowdsourced object detection model using an RCNN object detection model and a bayesian classifier combination network BCCNet (This main model we are comparing the results to)
- `YoloV5 IID`: This model doesn't have crowdsourcing functionality and is just YoloV5 object detection. The data to train this model has to be treated as ground truth data, so a separate dataset is used where different annotators for a single image are separated and treated as individual ground truth labels..
- `YoloV5 Agregation Preprocessing`: This algorithm agregates nearby labels of the same disease to create ground truth labels from crowdsourced labels (Not Machine learning). It will also use yoloV5 for the object detection `MIGHT NOT BE NEEDED`


`ALL MODELS HAVE HYPERPAMETER TUNING AND ARE TRAINED TO THE BEST OF OUR ABILITY`

|  | `Dataset 1 (All Annotators) [CSall]` | `Test Data (Ground Truth) [GTtest]` |
|---|---|---|
| `Crowd Yolo` | Expert and non-expert Crowdsourced data | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |
| `Crowd  RCNN` | Expert and non-expert Crowdsourced data | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |
| `YoloV5 IID` | Images are repeated and labels are separated to act as ground trouth labels for each radiograph (from same dataset) | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |
| `MIGHT NOT BE NEEDED` `YoloV5 Agregation Preprocessing` | Expert and non-expert Crowdsourced data agregated into ground truth labels | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |


## Experiment Set 2

There are 3 datasets used in these experiments:
- `Dataset 1 (CSall)`: Crowdsourced with both expert and non-expert labels 
- `Dataset 2 (CSexp)`: Crowdsourced (same dataset as above) with just expert labels filtered, no non-expert labels.  
- `Test Dataset (GTtest)`: Expert only ground truth labels with unseen data (small dataset completed by a single expert)

There are 4 models/techniques to expeiment:
- `Crowd Yolo`: A combination of YoloV5 object detection algorithm and a bayesian classifier combination network BCCNet (a network that decides the annotators value as an annotator inorder to train the yolo model). (main model the paper is about)
- `Crowd RCNN`: Another crowdsourced object detection model using an RCNN object detection model and a bayesian classifier combination network BCCNet (This main model we are comparing the results to)
- `YoloV5 IID`: This model doesn't have crowdsourcing functionality and is just YoloV5 object detection. The data to train this model has to be treated as ground truth data, so a separate dataset is used where different annotators for a single image are separated and treated as individual ground truth labels..
- `YoloV5 Agregation Preprocessing`: This algorithm agregates nearby labels of the same disease to create ground truth labels from crowdsourced labels (Not Machine learning). It will also use yoloV5 for the object detection `MIGHT NOT BE NEEDED`


`ALL MODELS HAVE HYPERPAMETER TUNING AND ARE TRAINED TO THE BEST OF OUR ABILITY`

|  | `Dataset 1 (All Annotators) [CSall]` | `Dataset 2 (Expert Labels Only) [CSexp]` | `Test Data (Ground Truth) [GTtest]` |
|---|---|---|---|
| `Crowd Yolo` | Expert and non-expert Crowdsourced data | Expert Crowdsourced data | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |
| `Crowd  RCNN` | Expert and non-expert Crowdsourced data | Expert Crowdsourced data | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |
| `YoloV5 IID` | Images are repeated and labels are separated to act as ground trouth labels for each radiograph (from same dataset) | Images are repeated and labels are separated to act as ground trouth labels only for expert annotator labels for each radiograph (from same dataset) | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |
| `MIGHT NOT BE NEEDED` `YoloV5 Agregation Preprocessing` | Expert and non-expert Crowdsourced data agregated into ground truth labels | Expert only Crowdsourced data agregated into ground truth labels | `Expert ground truth unseen data. Used as the test dataset for both train datasets.` |

LAST ONE MIGHT NOT BE NEEDED

## Installs

#### Please run the 'Install Requirements.ipynb' file in the base folder

## Libraries 

In [1]:
import os
import sys
import json
import yaml
from collections import defaultdict
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as mpatch
import matplotlib
sys.path.append('../src/cyolo_utils')
from results_plotter import read_exp_results as read_exp

### Choice of Hyper Parameters

Below are all the inital parameters you can change.



| `Parameter` | `Description` | `Type` | `Default` |
| --- | --- | --- | --- |
| bcc_epoch | start-epoch for BCC+YOLO run; use -1 for no BCC. | int | 0 |
| qtfilter_epoch | start-epoch for qt-filter; use -1 for no qt-filter. | int | -1 |
| qt_thres_mode | one of '', 'conf-count', 'entropy', 'conf-val' | str | '' |
| qt_thres | the threshold value. | float | 0.0 |
| hybrid_entropy_thres | the entropy threshold value (only to be used when running the hybrid filter). | float | 0.0 |
| hybrid_conf_thres | the confidence threshold value (only to be used when running the hybrid filter). | float | 0.0 |
| weights | initial weights path | str | 'yolov5s.pt' |
| cfg | model.yaml path | str | '' |
| data | dataset.yaml path | str | '../data/coco128.yaml' |
| hyp | hyperparameters path | str | '../data/hyps/hyp.scratch.yaml' |
| epochs | number of epoch repeates | int | 300 |
| batch-size | total batch size for all GPUs | int | 16 |
| imgsz | train, val image size (pixels) | int | 640 |
| rect | rectangular training | call | store_true |
| resume | resume most recent training | bool | False |
| nosave | only save final checkpoint | call | store_true |
| noval | only validate final epoch | call | store_true |
| noautoanchor | disable autoanchor check | call | store_true |
| evolve | evolve hyperparameters for x generations | int | const:300 |
| bucket | gsutil bucket | str | '' |
| cache | --cache images in "ram" (default) or "disk" | str | const:'ram' |
| image-weights | use weighted image selection for training | call | store_true |
| device | cuda device, i.e. 0 or 0,1,2,3 or cpu | call | store_true |
| multi-scale | vary img-size +/- 50%% | call | store_true |
| single-cls | train multi-class data as single-class | call | store_true |
| adam | use torch.optim.Adam() optimizer | call | store_true |
| sync-bn | use SyncBatchNorm, only available in DDP mode | call | store_true |
| workers | maximum number of dataloader workers | int | 8 |
| project | save to project/name | N/A | runs/train |
| entity | W&B entity | N/A | None |
| name | save to project/name | N/A | 'exp' |
| exist-ok | existing project/name ok, do not increment | call | store_true |
| quad | quad dataloader | call | store_true |
| linear-lr | linear LR | call | store_true |
| label-smoothing | Label smoothing epsilon | float | 0.0 |
| upload_dataset | Upload dataset as W&B artifact table | call | store_true |
| bbox_interval | Set bounding-box image logging interval for W&B | int | -1 |
| save_period | Log model after every "save_period" epoch | int | -1 |
| artifact_alias | version of dataset artifact to be used | str | latest |
| local_rank | DDP parameter, do not modify | int | -1 |
| freeze | Number of layers to freeze. backbone=10, all=24 | int | 0 |
| patience | EarlyStopping patience (epochs) | int | 1100 |


# EXPERIMENT SET 1 

# Experiment 1 (Crowd Yolo Model)

### This section runs the experiments, prints results, and plots graphs for both dataset using the Crowd Yolo model.

In [ ]:
!pip install -r yolov5_master/requirements.txt
!pip install python-utils
!pip install tensorflow

In [1]:
import os
import torch
import wandb
#!pip install seaborn
#!pip install torch
#!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html --user
#!wandb login --host=https://api.wandb.ai
#!wandb login --relogin b0d31701f11706105f7eec6a2d5cff4f9e89b18c

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%run -i ../src/train.py --data ../data/cyolo.yaml --batch-size 20 --epochs 80 --bcc_epoch 0 --name Expset1_all_cyolo

YOLOv5  2022-1-18 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, obj=1.0, cls=0.5, cls_pw=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir runs\train', view at http://localhost:6006/


train: bcc_epoch=0, qtfilter_epoch=-1, qt_thres_mode=, qt_thres=0.0, hybrid_entropy_thres=0.0, hybrid_conf_thres=0.0, weights=yolov5s.pt, cfg=, data=../data/cyolo.yaml, hyp=../data/hyps/hyp.scratch.yaml, epochs=80, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=Expset1_all_cyolo, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=1100
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


wandb: Currently logged in as: dddetection (use `wandb login --relogin` to force relogin)


Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

Plotting labels... 


...


     Epoch   gpu_mem       box       obj       cls    labels  img_size

YOLO+BCC
  0%|          | 0/4 [00:00<?, ?it/s]

Minimum probs (c1, c2, bkgd): [2e-06, 1e-06, 0.140625]
Maximum probs (c1, c2, bkgd): [0.786522, 0.133228, 1.0]


     33/79     13.1G   0.06422   0.07698   0.01153       222       640:  25%|██▌       | 1/4 [00:00<00:01,  1.69it/s]


Minimum probs (c1, c2, bkgd): [1e-06, 3e-06, 0.141602]
Maximum probs (c1, c2, bkgd): [0.82282, 0.095168, 1.0]


KeyboardInterrupt: 

In [3]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
!python ../src/val.py --weights runs/train/Expset1_all_cyolo/weights/best.pt --data ../data/iid-tvt.yaml --task test --name Expset1_all_cyolo_test

val: data=../data/iid-tvt.yaml, weights=['runs/train/Expset1_all_cyolo/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=Expset1_all_cyolo_test, exist_ok=False, half=False
                 all         10         32      0.566      0.182     0.0328    0.00475
           bone-loss         10         11      0.133      0.364     0.0649    0.00938
       dental-caries         10         21          1          0    0.00063   0.000126
Speed: 1.6ms pre-process, 24.4ms inference, 3.9ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs\val\Expset1_all_cyolo_test


YOLOv5  2022-1-18 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

Fusing layers... 
C:\Users\rb01243\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

test: Scanning '..\data\datasets\iid-tvt\labels\test.cache' images and labels... 10 found, 0 missing, 0 empty, 0 corrupted: 100%|##########| 10/10 [00:00<?, ?it/s]
test: Scanning '..\data\datasets\iid-tvt\labels\test.cache' images and labels... 10 found, 0 missing, 0 empty, 0 corrupted: 100%|##########| 10/10 [00:00<?, ?it/s]

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0%|          | 0/1 [00:00<?, ?it/s]
               Class     Images     Labels          P 

In [2]:
import os
import torch
import wandb
# PROJ_PATH = '..'
# sys.path.append(PROJ_PATH)
# from src.data_preparer import prepare_data
#os.chdir(origDir)
#os.chdir(os.getcwd()+'/yolov5_master')

#%run -i train.py --data ../data/iid-tvt.yaml --batch-size 20 --epochs 18 --weights models/yolov5s.pt
#%run -i src/train.py --data data/iid-tvt.yaml --batch-size 20 --epochs 18 --bcc_epoch -1 --cfg src/models/yolov5s.yaml
#%run -i yolov5_master/train.py --data data/iid-tvt.yaml --batch-size 20 --epochs 18 --cfg yolov5_master/models/yolov5s.yaml
#%run -i train.py --data data/iid-tvt.yaml --batch-size 20 --epochs 18 --weights models/yolov5s.pt --bcc_epoch -1 yolov5_master/train.py yolov5_master/models/yolov5s.pt

#os.chdir(origDir) 

# SRC_PATH = os.path.join(PROJ_PATH, 'src')
# DATA_PATH = os.path.join(PROJ_PATH, 'data/datasets')

# # data_modes = ['sty', 'stcy']
# # train_ratios = (0.7, 0.2, 0.1)
# # data_path = DATA_PATH
# # for mode in data_modes:
# #     prepare_data(mode, train_ratios, data_path)


# data = '../data/iid-tvt.yaml'
# #data = '../data/cyolo.yaml'
# batch_size = 20 # Change this to number of train images
# epochs = 20
# bcc_epoch = -1 # Involve BCC from epoch number "bcc_epoch". Set to -1 for no BCC. 0 for all BCC.
# exp_num = 10002
# name = f'exp{exp_num}'

# print(SRC_PATH)
#wandb.init(project="cyolo", entity="dddetection")
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#torch. set_printoptions(profile="full")
#torch. set_printoptions(profile="default")
%run -i ../src/train.py --data ../data/iid-tvt.yaml --batch-size 20 --epochs 20 --bcc_epoch -1 --name iid
#%run -i ../src/train.py --data ../data/coco2.yaml --batch-size 18 --epochs 2 --bcc_epoch -1 --name testCOCO1

# !python $SRC_PATH/train.py --data $data \
#                            --batch-size $batch_size \
#                            --epochs $epochs \
#                            --bcc_epoch $bcc_epoch \
#                            --name $name \
#                            --exist-ok

ImportError: cannot import name 'save_one_box' from 'utils.general' (C:\Users\rb01243\OneDrive - University of Surrey\Documents\GitHub\dental_disease\yolov5-master\utils\general.py)

## Analysis

{ analysis goes here }

# Experiment 2 (Crowd RCNN)

### This section runs the experiments, prints results, and plots graphs for both dataset using the Crowd RCNN model.

In [ ]:
%run -i ../Crowd-R-CNN-master/trainval_net.py --dataset ../data/datasets/bcc-tvt/ --batch_size 20 --epochs 80 --name Expset1_all_RCNN

## Analysis

{ analysis goes here }

# Experiment 3 (YoloV5 IID Annotators)

### This section runs the experiments, prints results, and plots graphs for all annotators where their labels are treated as ground truth labels, using the YoloV5 model.

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%run -i ../src/train.py --data ../data/iid-tvt.yaml --batch-size 20 --epochs 80 --bcc_epoch -1 --name Expset1_iid_yolov5

YOLOv5  2022-1-18 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, obj=1.0, cls=0.5, cls_pw=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir runs\train', view at http://localhost:6006/


train: bcc_epoch=-1, qtfilter_epoch=-1, qt_thres_mode=, qt_thres=0.0, hybrid_entropy_thres=0.0, hybrid_conf_thres=0.0, weights=yolov5s.pt, cfg=, data=../data/iid-tvt.yaml, hyp=../data/hyps/hyp.scratch.yaml, epochs=80, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=Expset1_iid_yolov5, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=1100
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


wandb: Currently logged in as: dddetection (use `wandb login --relogin` to force relogin)


Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

Plotting labels... 


...

                 all         80        220      0.658      0.203      0.166      0.052

Evaluating pycocotools mAP... saving runs\train\Expset1_iid_yolov5\_predictions.json...
requirements: pycocotools not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pycocotools'' returned non-zero exit status 1.
pycocotools unable to run: No module named 'pycocotools'


Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 2/2 [00:00<00:00,  4.15it/s]


                 all         80        220      0.709      0.173       0.17     0.0547

Evaluating pycocotools mAP... saving runs\train\Expset1_iid_yolov5\_predictions.json...
requirements: pycocotools not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pycocotools'' returned non-zero exit status 1.
pycocotools unable to run: No module named 'pycocotools'
Optimizer stripped from runs\train\Expset1_iid_yolov5\weights\last.pt, 14.3MB
Optimizer stripped from runs\train\Expset1_iid_yolov5\weights\best.pt, 14.3MB


metrics/mAP_0.5,▁▁▁▂▃▄▄▅▃▄▄▅▄▅▅▆▅▆▆▆▆▆▇▆▆▆▇▆▇▇█▇▇▇▇▇▇█▇▇
metrics/mAP_0.5:0.95,▁▁▁▂▂▃▃▄▂▃▃▄▃▄▄▅▅▅▅▆▅▆▆▅▅▅▆▆▇▇█▆▇▇▇█▇█▇█
metrics/precision,▁▁▁▇▇▇▇█▇▇▇▇▇▇▇█▇▇▇█▇██▇█▇██▇██▇▇███████
metrics/recall,▁▃▂▃▅▅▆▅▄▅▄▇▄▆█▄█▇█▆█▅▅█▅▇▆▅▇▆▆██▅▆▆▆▆▆▆
train/box_loss,█▆▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train/cls_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/obj_loss,▆▇██▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/box_loss,█▆▅▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/cls_loss,█▆▄▃▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/obj_loss,▇██▆▄▃▃▂▁▂▃▂▁▂▁▁▂▁▂▄▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄
x/lr0,▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇██████████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅


Results saved to runs\train\Expset1_iid_yolov5


In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
!python ../src/val.py --weights runs/train/Expset1_iid_yolov5/weights/best.pt --data ../data/iid-tvt.yaml --task test --name Expset1_iid_yolov5_test

val: data=../data/iid-tvt.yaml, weights=['runs/train/Expset1_iid_yolov5/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=Expset1_iid_yolov5_test, exist_ok=False, half=False
                 all         10         32      0.749     0.0455     0.0466     0.0144
           bone-loss         10         11      0.498     0.0909     0.0909     0.0283
       dental-caries         10         21          1          0    0.00228   0.000457
Speed: 0.7ms pre-process, 3.0ms inference, 2.0ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs\val\Expset1_iid_yolov5_test


YOLOv5  2022-1-18 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

Fusing layers... 
C:\Users\rb01243\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

test: Scanning '..\data\datasets\iid-tvt\labels\test.cache' images and labels... 10 found, 0 missing, 0 empty, 0 corrupted: 100%|##########| 10/10 [00:00<?, ?it/s]
test: Scanning '..\data\datasets\iid-tvt\labels\test.cache' images and labels... 10 found, 0 missing, 0 empty, 0 corrupted: 100%|##########| 10/10 [00:00<?, ?it/s]

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0%|          | 0/1 [00:00<?, ?it/s]
               Class     Images     Labels          P 

## Analysis

{ analysis goes here }

# Experiment 4 (YoloV5 Agregation Preprocessing)

### This section runs the experiments, prints results, and plots graphs for both dataset where ground truth labels are agregated into ground truth labels. This experiment also uses YoloV5 as the object detection algorithm

## 4.2  CSexp (Dataset 2)

## Analysis

{ analysis goes here }

# Conclusion Experiment Set 1

# EXPERIMENT SET 2 

# Experiment 1 (Crowd Yolo Model)

### This section runs the experiments, prints results, and plots graphs for both dataset using the Crowd Yolo model.

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%run -i ../src/train.py --data ../data/All_Volunteers_Calc_Removed_Crowdsourced_IID_Val.yaml --batch-size 25 --epochs 50 --bcc_epoch 0 --device 0 --name ex1_Zoon_with_Calc
#--batch-size 100

YOLOv5  2022-2-8 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, obj=1.0, cls=0.5, cls_pw=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir runs\train', view at http://localhost:6006/


train: bcc_epoch=0, confusion_matrix_diagonal_prior_hyp=0.1, convergence_threshold_hyp=1e-06, qtfilter_epoch=-1, qt_thres_mode=, qt_thres=0.0, hybrid_entropy_thres=0.0, hybrid_conf_thres=0.0, weights=yolov5s.pt, cfg=, data=../data/All_Volunteers_Crowdsourced.yaml, hyp=../data/hyps/hyp.scratch.yaml, epochs=50, batch_size=25, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=ex1_Zoon_with_Calc, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=1100
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
device:  cuda:0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dddetection (use `wandb login --relogin` to force relogin)


Overriding model.yaml nc=80 with nc=3

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

Augmentation is:  False


...

                 all       1900          0          0          0          0          0


               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 63/63 [00:06<00:00,  9.25it/s]


                 all       1556          0          0          0          0          0



     Epoch   gpu_mem       box       obj       cls    labels  img_size

YOLO+BCC
     19/49       14G         0 4.295e-05         0       323       640:  86%|████████▌ | 54/63 [03:50<00:38,  4.27s/it]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%run -i ../src/train.py --data ../data/All_Volunteers_Calc_Removed_Crowdsourced.yaml --batch-size 25 --epochs 10 --bcc_epoch 0 --device 0 --name ex1_Zoon_without_Calc_10epo
#--batch-size 100

YOLOv5  2022-2-8 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, obj=1.0, cls=0.5, cls_pw=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir runs\train', view at http://localhost:6006/


train: bcc_epoch=0, confusion_matrix_diagonal_prior_hyp=0.1, convergence_threshold_hyp=1e-06, qtfilter_epoch=-1, qt_thres_mode=, qt_thres=0.0, hybrid_entropy_thres=0.0, hybrid_conf_thres=0.0, weights=yolov5s.pt, cfg=, data=../data/All_Volunteers_Calc_Removed_Crowdsourced.yaml, hyp=../data/hyps/hyp.scratch.yaml, epochs=10, batch_size=25, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=ex1_Zoon_without_Calc_10epo_Test_Val_CS, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=1100
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
device:  cuda:0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dddetection (use `wandb login --relogin` to force relogin)


Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

Augmentation is:  False


...

                 all        388       4923     0.0109   0.000605    0.00789    0.00117

Evaluating pycocotools mAP... saving runs\train\ex1_Zoon_without_Calc_10epo_Test_Val_CS\_predictions.json...
requirements: pycocotools not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pycocotools'' returned non-zero exit status 1.
pycocotools unable to run: No module named 'pycocotools'


Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 8/8 [00:01<00:00,  7.05it/s]


                 all        388       4923     0.0109   0.000605    0.00789    0.00117

Evaluating pycocotools mAP... saving runs\train\ex1_Zoon_without_Calc_10epo_Test_Val_CS\_predictions.json...
requirements: pycocotools not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pycocotools'' returned non-zero exit status 1.
pycocotools unable to run: No module named 'pycocotools'
Optimizer stripped from runs\train\ex1_Zoon_without_Calc_10epo_Test_Val_CS\weights\last.pt, 14.3MB
Optimizer stripped from runs\train\ex1_Zoon_without_Calc_10epo_Test_Val_CS\weights\best.pt, 14.3MB


metrics/mAP_0.5,▁▂▂▃▃▂▁▃▇█
metrics/mAP_0.5:0.95,▁▂▃▄▃▃▁▄▆█
metrics/precision,▁▃▃▄▄▃▁▄▆█
metrics/recall,█▅█▃▂▁▁▁▁▁
train/box_loss,▁▁▁▁▁▁▁▁▁▁
train/cls_loss,▁▁▁▁▁▁▁▁▁▁
train/obj_loss,█▃▂▂▁▁▁▁▁▁
val/box_loss,▁▇████▇█▇▇
val/cls_loss,▁▄█▅▅▆▆█▇▆
val/obj_loss,▁▁▃▄▅▆▇▇██
x/lr0,▁▄▆▇██▇▆▅▄


Results saved to runs\train\ex1_Zoon_without_Calc_10epo_Test_Val_CS


## 1.1  CSall (Dataset 1)

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%run -i ../src/train.py --data ../data/All_Volunteers_Crowdsourced.yaml --batch-size 20 --epochs 100 --bcc_epoch 0 --device 0 --name LargeDS_cyolo
#--batch-size 100

YOLOv5  2022-2-8 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, obj=1.0, cls=0.5, cls_pw=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir runs\train', view at http://localhost:6006/


train: bcc_epoch=0, confusion_matrix_diagonal_prior_hyp=0.1, convergence_threshold_hyp=1e-06, qtfilter_epoch=-1, qt_thres_mode=, qt_thres=0.0, hybrid_entropy_thres=0.0, hybrid_conf_thres=0.0, weights=yolov5s.pt, cfg=, data=../data/All_Volunteers_Crowdsourced.yaml, hyp=../data/hyps/hyp.scratch.yaml, epochs=10, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=LargeDS_cyolo, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=1100
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
device:  cuda:0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dddetection (use `wandb login --relogin` to force relogin)


Overriding model.yaml nc=80 with nc=3

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

Plotting labels... 


Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\LargeDS_cyolo4
Starting training for 10 epochs...

     Epoch   gpu_mem       box       obj       cls    labels  img_size

YOLO+BCC



autoanchor: Analyzing anchors... anchors/target = 5.18, Best Possible Recall (BPR) = 0.9986


       0/9     11.2G         0   0.01272         0       239       640:  72%|███████▏  | 63/88 [10:04<03:59,  9.59s/it]


KeyboardInterrupt: 

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
!python ../src/val.py --weights runs/train/LargeDS_cyolo/weights/best.pt --data ../data/All_Volunteers_Crowdsourced.yaml --task test --name LargeDS_cyolo_test

val: data=../data/All_Volunteers_Crowdsourced.yaml, weights=['runs/train/Expset2_all_cyolo136/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=Expset2_all_cyolo136_test, exist_ok=False, half=False
                 all        100        432    0.00203     0.0149    0.00103   0.000154
     calculus-plaque        100        316    0.00251     0.0127    0.00127    0.00019
       dental-caries        100        116    0.00155     0.0172   0.000786   0.000118
Speed: 0.7ms pre-process, 6.1ms inference, 4.5ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs\val\Expset2_all_cyolo136_test


YOLOv5  2022-1-18 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

Fusing layers... 
C:\Users\rb01243\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs

test: Scanning '..\data\datasets\All_Volunteers_Crowdsourced\labels\test' images and labels...:   0%|          | 0/100 [00:00<?, ?it/s]
test: Scanning '..\data\datasets\All_Volunteers_Crowdsourced\labels\test' images and labels...1 found, 0 missing, 0 empty, 0 corrupted:   1%|1         | 1/100 [00:03<05:10,  3.14s/it]
test: Scanning '..\data\datasets\All_Volunteers_Crowdsourced\labels\test' images and labels...92 found, 0 missing, 0 empty, 0 corrupted:  92%|#########2| 92/100 [00:03<00:00, 39.92it/s]
tes

## 1.2  CSexp (Dataset 2)

## Analysis

{ analysis goes here }

# Experiment 2 (Crowd RCNN)

### This section runs the experiments, prints results, and plots graphs for both dataset using the Crowd RCNN model.

## 2.1  CSall (Dataset 1)

## 2.2  CSexp (Dataset 2)

## Analysis

{ analysis goes here }

# Experiment 3 (YoloV5 IID)

### This section runs the experiments, prints results, and plots graphs for ranomly selected ground truth labels for both dataset using the YoloV5 model.

## 3.1  CSall (Dataset 1)

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%run -i ../src/train.py --data ../data/All_Volunteers_IID.yaml --batch-size 20 --epochs 20 --device 0 --name testIIDZOON

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dddetection (use `wandb login --relogin` to force relogin)
train: weights=..\yolov5-master\yolov5s.pt, cfg=, data=../data/All_Volunteers_IID.yaml, hyp=..\yolov5-master\data\hyps\hyp.scratch.yaml, epochs=20, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=..\yolov5-master\runs\train, name=testIIDZOON, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  2022-2-8 torch 1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


Overriding model.yaml nc=80 with nc=3

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

module 'signal' has no attribute 'SIGALRM'


      0/19     4.33G   0.09036   0.04595   0.03228        99       640: 100%|██████████| 426/426 [02:00<00:00,  3.54it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 24/24 [00:06<00:00,  3.53it/s]
                 all        946       2967     0.0429      0.231     0.0272    0.00621

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      1/19     5.69G   0.07681   0.04775   0.02225        57       640: 100%|██████████| 426/426 [01:58<00:00,  3.61it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 24/24 [00:07<00:00,  3.07it/s]
                 all        946       2967       0.11      0.105     0.0384     0.0096

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      2/19     5.69G   0.07455   0.04713   0.01962        91       640: 100%|██████████| 426/426 [01:58<00:00,  3.60it/s]
               Class     Images     Labels  

Optimizer stripped from ..\yolov5-master\runs\train\testIIDZOON2\weights\last.pt, 14.4MB
Optimizer stripped from ..\yolov5-master\runs\train\testIIDZOON2\weights\best.pt, 14.4MB


Model Summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 24/24 [00:08<00:00,  2.95it/s]
                 all        946       2967      0.138      0.184     0.0791     0.0221
     calculus-plaque        946        586      0.139      0.142     0.0791     0.0227
       dental-caries        946       1239      0.148      0.148     0.0744     0.0225
           bone-loss        946       1142      0.127      0.263     0.0838      0.021


metrics/mAP_0.5,▁▃▄▅▆▆▆▇▇▇▇▇▇████████
metrics/mAP_0.5:0.95,▁▂▄▅▅▆▆▇▆▆▆▆▇██████▇█
metrics/precision,▁▄▄▄▅▅▆▅▅▆▆▅▆██▇▅▆▆▇▆
metrics/recall,█▁▄▄▄▄▄▅▄▅▄▄▅▄▄▄▆▅▅▅▅
train/box_loss,█▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
train/cls_loss,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/obj_loss,▁▇▅█▆█▆▇▇▇▇▆▆▇▇▇▇▇▆█
val/box_loss,█▇▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val/cls_loss,█▅▃▂▂▂▂▂▁▁▁▁▂▁▂▁▁▂▂▂▂
val/obj_loss,▁█▄▂▅▃▅▂▃▄▃▄▂▃▃▃▂▃▂▃▂
x/lr0,▃▅███▇▇▇▆▆▅▅▄▃▃▂▂▂▁▁


Results saved to ..\yolov5-master\runs\train\testIIDZOON2


## 3.2  CSexp (Dataset 2)

## Analysis

{ analysis goes here }

# Experiment 4 (YoloV5 Agregation Preprocessing)

### This section runs the experiments, prints results, and plots graphs for both dataset where ground truth labels are agregated into ground truth labels. This experiment also uses YoloV5 as the object detection algorithm

## 4.1  CSall (Dataset 1)

## 4.2  CSexp (Dataset 2)

## Analysis

{ analysis goes here }

# Conclusion Experiment Set 2